In [1]:
# imoprt required library
import requests                 # Request HTTP library
import urllib3                  # HTTP library with thread safe connection
import numpy as np              # For numerical operations
import pandas as pd             # For data manipulation and analysis
from bs4 import BeautifulSoup   # HTML parsing
import matplotlib.pyplot as plt # For Data visaulization

%matplotlib inline

In [2]:
# Disable warnings
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# Base scrapeme shop URL
scrapme = "https://scrapeme.live/shop/"

In [3]:
# Request page without verification
scrapme_response = requests.get(scrapme, verify=False)
scrapme_response

<Response [200]>

In [4]:
scrapme_response.text

'\n<!doctype html>\n<html lang="en-GB">\n<head>\n<meta charset="UTF-8">\n<meta name="viewport" content="width=device-width, initial-scale=1, maximum-scale=2.0">\n<link rel="profile" href="http://gmpg.org/xfn/11">\n<link rel="pingback" href="https://scrapeme.live/xmlrpc.php">\n\n<title>Products &#8211; ScrapeMe</title>\n<link rel=\'dns-prefetch\' href=\'//fonts.googleapis.com\' />\n<link rel=\'dns-prefetch\' href=\'//s.w.org\' />\n<link rel="alternate" type="application/rss+xml" title="ScrapeMe &raquo; Feed" href="https://scrapeme.live/feed/" />\n<link rel="alternate" type="application/rss+xml" title="ScrapeMe &raquo; Comments Feed" href="https://scrapeme.live/comments/feed/" />\n<link rel="alternate" type="application/rss+xml" title="ScrapeMe &raquo; Products Feed" href="https://scrapeme.live/shop/feed/" />\n\t\t<script type="text/javascript">\n\t\t\twindow._wpemojiSettings = {"baseUrl":"https:\\/\\/s.w.org\\/images\\/core\\/emoji\\/11\\/72x72\\/","ext":".png","svgUrl":"https:\\/\\/s.w

In [5]:
# Parse HTML
scrapme_soup = BeautifulSoup(scrapme_response.text, "html.parser")

In [6]:
print(scrapme_soup.prettify())

<!DOCTYPE html>
<html lang="en-GB">
 <head>
  <meta charset="utf-8"/>
  <meta content="width=device-width, initial-scale=1, maximum-scale=2.0" name="viewport"/>
  <link href="http://gmpg.org/xfn/11" rel="profile"/>
  <link href="https://scrapeme.live/xmlrpc.php" rel="pingback"/>
  <title>
   Products – ScrapeMe
  </title>
  <link href="//fonts.googleapis.com" rel="dns-prefetch">
   <link href="//s.w.org" rel="dns-prefetch">
    <link href="https://scrapeme.live/feed/" rel="alternate" title="ScrapeMe » Feed" type="application/rss+xml"/>
    <link href="https://scrapeme.live/comments/feed/" rel="alternate" title="ScrapeMe » Comments Feed" type="application/rss+xml"/>
    <link href="https://scrapeme.live/shop/feed/" rel="alternate" title="ScrapeMe » Products Feed" type="application/rss+xml"/>
    <script type="text/javascript">
     window._wpemojiSettings = {"baseUrl":"https:\/\/s.w.org\/images\/core\/emoji\/11\/72x72\/","ext":".png","svgUrl":"https:\/\/s.w.org\/images\/core\/emoji\/11\

In [7]:
# Title of page
scrapme_soup.title

<title>Products – ScrapeMe</title>

In [8]:
# Title of page as string
scrapme_soup.title.string

'Products – ScrapeMe'

In [9]:
# Find all products in the oage
scrapme_products = scrapme_soup.find_all("li", {'class' : "product"})

In [10]:
scrapme_products

[<li class="post-759 product type-product status-publish has-post-thumbnail product_cat-pokemon product_cat-seed product_tag-bulbasaur product_tag-overgrow product_tag-seed first instock sold-individually taxable shipping-taxable purchasable product-type-simple">
 <a class="woocommerce-LoopProduct-link woocommerce-loop-product__link" href="https://scrapeme.live/shop/Bulbasaur/"><img alt="" class="attachment-woocommerce_thumbnail size-woocommerce_thumbnail wp-post-image" height="324" sizes="(max-width: 324px) 100vw, 324px" src="https://scrapeme.live/wp-content/uploads/2018/08/001-350x350.png" srcset="https://scrapeme.live/wp-content/uploads/2018/08/001-350x350.png 350w, https://scrapeme.live/wp-content/uploads/2018/08/001-150x150.png 150w, https://scrapeme.live/wp-content/uploads/2018/08/001-300x300.png 300w, https://scrapeme.live/wp-content/uploads/2018/08/001-100x100.png 100w, https://scrapeme.live/wp-content/uploads/2018/08/001-250x250.png 250w, https://scrapeme.live/wp-content/uploa

In [11]:
# Define initial list named 'data'
data = []

# Loop through pages
for page in range(1, 49):
    if page == 1:
        page_link = "https://scrapeme.live/shop/"
    else:
        page_link = f"https://scrapeme.live/shop/page/{page}/"

    scrap_response = requests.get(page_link, verify=False)
    # HTML parser
    scrap_soup = BeautifulSoup(scrap_response.text, "html.parser")

    # Find all products
    scrap_products = scrap_soup.find_all("li", {"class": "product"})

    # Loop through products
    for product in scrap_products:
        # Excract the product name
        name = product.find("h2", {'class':"woocommerce-loop-product__title"}).text
        # Excract the product price
        price = product.find("span",{'class':"price"}).text
        # Excract the product link
        link = product.find("a", {'class':"woocommerce-LoopProduct-link woocommerce-loop-product__link"})["href"]

        # Enter the product's page
        products_page = requests.get(link, verify=False)
        products_soup = BeautifulSoup(products_page.text, "html.parser")

        # Excract the description of product
        description = products_soup.find("div", {'class':"woocommerce-product-details__short-description"}).text

        # Excract the product sku
        sku = products_soup.find("span", {'class':"sku"}).text

        # Excract the product category
        category = products_soup.find("span", {'class':"posted_in"}).text

        # Excract the product tags
        tags = products_soup.find("span", {'class':"tagged_as"}).text

        # Excract the product stock
        stock = products_soup.find("p", {'class':"stock"}).text

        data.append({
            "Name": name,
            "Price": price,
            "Link": link,
            "Description": description,
            "SKU": sku,
            "Category": category,
            "Tags": tags,
            "Stock": stock
        })


In [12]:
df = pd.DataFrame(data)
df.to_csv("scrapeme_products.csv", index=False)